In [1]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')


/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_17324/3758130481.py:49: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [ ]:
#BIG DATASET

large_df_combined = load_csv_dataset('updated COMBINED DF_WORKING.csv')


terrell_new_df = load_csv_dataset('CSVs of edge cases - for T/updated Terrell new DF.csv')
len(terrell_new_df), len(large_df_combined)


In [ ]:
#Group 1 

group_1_updated = load_csv_dataset('updated_Group1_fixes_complete_ii.csv')
len(group_1_updated)

In [ ]:
#Group 2 

online_lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Online lookup required - 1 name found.csv')

lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/lookup_required_matches.csv')

cyrillic_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Cyrillic_Found_Guys.csv')
len(online_lookup_group_2), len(lookup_group_2), len(cyrillic_group_2)

In [ ]:
#Group 3 

cyrillic_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Cyrillic_Guys_Nobody_Found.csv')

zeros_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/odd_name_found_0_cases.csv')

online_lookup_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Online lookup required - 0 names found.csv')

len(cyrillic_group_3), len(zeros_group_3), len(online_lookup_group_3)

In [ ]:
leagues_value

In [ ]:
# Other group

wrong_df = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated Terrell lookup cases - wrong DF.csv')
wrong_df = wrong_df.drop(columns={'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'})
lookup_required_dudes = load_csv_dataset('CSVs of edge cases - for F/"other" group/lookup_required_dudes.csv')

large_dataset_said_0 = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated_large_dataset_said_0.csv')

match_date_filter_said_0_guys = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated match date said zero guys.csv')

len(wrong_df), len(lookup_required_dudes), len(large_dataset_said_0), len(match_date_filter_said_0_guys)

## WORKING WITH WRONG DF

In [2]:
from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")


        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

In [ ]:
wrong_df['New Found Name'] = ''
wrong_df['Status II'] = ''

In [ ]:
wrong_df[wrong_df['Status'] == 'SUCCESS']['Lookup Return Case'].value_counts()

In [ ]:
success_status_counter = 0
for i in range(0, 1063):
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = wrong_df.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']

    if type(row['Found Name']) == str:
        if row['Found Name'].startswith('['):
            FOUND_NAME = eval(row['Found Name']) 
        else:
            FOUND_NAME = row['Found Name']
    
    if row['Status II'] != 'Done':
    
        if row['Status'] == 'SUCCESS':
            #for all the success guys. 
            #these are all people that eventually T needs to use imputing methods.   
            NAMES_FOUND = row['Name(s) Found']

            ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
            if ORIGINAL_JERSEY == 'evčík':
                ORIGINAL_JERSEY = 'P Ševčík'
            this_country_code = row['Team Country Code']
            if pd.isna(this_country_code):
                country_name = 'Namibia'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
            dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
            if 'Mohd' in ORIGINAL_JERSEY:
                ORIGINAL_JERSEY = ORIGINAL_JERSEY.replace('Mohd', 'Mohammed')
            if type(FOUND_NAME) == list:
                if NAMES_FOUND.startswith('['):
                    for name in eval(NAMES_FOUND):
                        FOUND_NAME.append(name)
                else:
                    FOUND_NAME.append(NAMES_FOUND)
                found_names = FOUND_NAME
            else:
                found_names = [FOUND_NAME, NAMES_FOUND]
            closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)

            threshold_last_name = threshold_player_match(ORIGINAL_JERSEY.split(' ')[-1], dataset_nationality)[0]
            if closest_name != []:
                intersection = [name for name in closest_name if name in found_names]
            else:
                intersection = [name for name in found_names if name in threshold_last_name]

            
            if list(set(intersection)) != []:
                if len(list(set(intersection))) == 1 and list(set(intersection)) != ['No matches found even with the lowest threshold.']:
                    if ORIGINAL_JERSEY == 'Mudir Al Radaei':
                        intersection = ['Mudir Abdurabu']
                    elif ORIGINAL_JERSEY == 'Noel-Mc Leod':
                        intersection = ['Kraig Noel-McLeod']
                    elif ORIGINAL_JERSEY == 'Nguyn Tin Duy':
                        intersection = ['Tien Duy Nguyen']
                    elif ORIGINAL_JERSEY == 'Abbas Al Hassan':
                        intersection = ['Abbas Al-Hassan']
                    elif ORIGINAL_JERSEY == 'Angel':
                        intersection = ['Wilker Ángel']

                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'intersection of lists is says{list(set(intersection))}')

                    success_status_counter += 1
                    wrong_df.at[i, 'New Found Name'] = list(set(intersection))[0]
                    wrong_df.at[i, 'Status II'] = 'Done - New'
                else:
                    #16 cases here left
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    # if type(filtered_names_match_date) == str:
                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
                    wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                    wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                    success_status_counter += 1
            else:
                0==0 #18 cases here
                success_status_counter += 1
                print(i, wrong_df.at[i, 'Date'], wrong_df.at[i, 'Nationality'], wrong_df.at[i, 'Match'], ORIGINAL_JERSEY, closest_name, found_names)
                print(i, f'lists is says{found_names}')
            
        else:
        
                      
            NAMES_FOUND = row['Name(s) Found']

            ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
            if ORIGINAL_JERSEY == 'evčík':
                ORIGINAL_JERSEY = 'P Ševčík'
            this_country_code = row['Team Country Code']
            if pd.isna(this_country_code):
                country_name = 'Namibia'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
            dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
            if 'Mohd' in ORIGINAL_JERSEY:
                ORIGINAL_JERSEY = ORIGINAL_JERSEY.replace('Mohd', 'Mohammed')
            if type(FOUND_NAME) == list:
                if NAMES_FOUND.startswith('['):
                    for name in eval(NAMES_FOUND):
                        FOUND_NAME.append(name)
                else:
                    FOUND_NAME.append(NAMES_FOUND)
                found_names = FOUND_NAME
            else:
                found_names = [FOUND_NAME, NAMES_FOUND]
            closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)
     
            if len(closest_name) == 1:
                print(i, ORIGINAL_JERSEY, [name for name in closest_name if name in found_names]) #closest_name, found_names
                success_status_counter += 1
                
                #wrong_df['Status 2'] MAKE A NEW STATUS TO MARK THEM OFF ONCE DONE  
            else:
                print(i)
                
                threshold_last_name = threshold_player_match(ORIGINAL_JERSEY.split(' ')[-1], dataset_nationality)[0]
                if closest_name != []:
                    intersection = [name for name in closest_name if name in found_names and name in threshold_last_name]
                else:
                    intersection = [name for name in found_names if name in threshold_last_name]
                if len(list(set(intersection))) == 2:
                    
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    if type(filtered_names_match_date) == str:
                        print(i, ORIGINAL_JERSEY, closest_name, found_names)
                        print(i, f'SINGLE - filtered bymatch date - intersection of lists is says{filtered_names_match_date}')
                        wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                        wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                    
                        success_status_counter += 1
                    elif type(filtered_names_match_date) == list:
                        0==0
                        print(i, ORIGINAL_JERSEY, closest_name, found_names)
                        print(i, f'MULTI - filtered bymatch date - intersection of lists is says{filtered_names_match_date}')
                        wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                        wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                        success_status_counter += 1
                else:
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
                    success_status_counter += 1
                    #wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                    #wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'


                #print(i, f"threshold match says {threshold_player_match(ORIGINAL_JERSEY, found_names)}")
print(success_status_counter)

In [ ]:
wrong_df.to_csv('wrong_df_corrected.csv')

## Working with Lookup Required Dudes

In [ ]:
lookup_required_dudes
#do the threshold match. if its either Name(s) found column or Found Name column use that name 

In [ ]:
for i in range(0, 2026):
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = lookup_required_dudes.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    this_country_code = row['Team Country Code']
    if pd.isna(this_country_code):
        country_name = 'Namibia'
    else:
        if this_country_code == 'BVI':
            country_name = 'British Virgin Islands'
        else:
            country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
    dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()

    matches_try_1 = threshold_player_match(ORIGINAL_JERSEY, dataset_nationality)[0]

    if len(matches_try_1) == 1:
        if row['Season'] <= 2012:
            0==0
        elif matches_try_1 == ['No matches found even with the lowest threshold.']:
            0==0
        else:
            print(i, f"jersey is {ORIGINAL_JERSEY}, country is {nationality}. match is {row['Match']} on {row['Date']}. matches is {matches_try_1}")
    

In [ ]:
leagues_value[leagues_value['Name'] == 'Donovan Ricketts']

## Match date said zero dudes

In [ ]:
match_date_filter_said_0_guys['Status New'] = ''

In [3]:
#FUNCTIONS

def find_transfermarkt_links(html_content):
    # Regular expression patterns for transfermarkt URLs
    transfermarkt_patterns = [
        r"transfermarkt\.de",
        r"transfermarkt\.us",
        r"transfermarkt\.world"
    ]
    transfermarkt_links = []

    # Find all 'a' tags in the HTML content
    links = html_content.find_all('a', href=True)

    # Filter links that match transfermarkt patterns
    for link in links:
        for pattern in transfermarkt_patterns:
            if re.search(pattern, link['href']):
                transfermarkt_links.append(link['href'])
                break

    return transfermarkt_links


def query_google_1(input_string):
    # Replace spaces with plus signs to format the query string
    query = '+'.join(input_string.split())

    # URL for Google search
    url = f"https://www.google.com/search?q={query}"

    # Send GET request to Google
    response = requests.get(url)

    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Return the parsed HTML content
        return soup
    else:
        # If request fails, print error message
        print("Failed to fetch results from Google.")
        return None
    

def extract_name_from_link(link):
        
    name_in_link = link.split('/')[3]
    name_in_link_tokens = name_in_link.split('-')
    capitalized_tokens = []

    for token in name_in_link_tokens:
        capitalized_tokens.append(token.capitalize())

    combined_string = ' '.join(capitalized_tokens)

    return combined_string


def process_google_query(input_text):
    html_result = query_google_1(input_text)
    if html_result:
        transfermarkt_links = find_transfermarkt_links(html_result)
        if transfermarkt_links:
            names_links = []
            for link in transfermarkt_links:
                if link.startswith('/url?q='):
                    link = link.split('/url?q=')[1]
                name_from_link = extract_name_from_link(link)
                names_links.append(name_from_link)
            if len(names_links) == 1:
                name = names_links[0]
                return name
            elif len(names_links) >= 2:
                name_list_matches = list(set(names_links))
                return name_list_matches
            else:
                return 0  # Nobody found
        else:
            return 0  # No links found
    else:
        return 0  # No HTML result

In [4]:
leagues_value_old = load_csv_dataset('MarketValues.csv')
leagues_value_old = leagues_value_old[leagues_value_old['Season'] <= 2013]
match_date_filter_said_0_guys = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated_match_date_filter_said_0_guys_ii.csv')

In [5]:
success_status_counter = 0
cases_2011_12 = 0
done_cases = 0
zero_cases = 0
multi_cases = 0
for i in range(0, 438):
    index = i
    row = match_date_filter_said_0_guys.loc[index]
    if row['Status New'] == 'Done' or row['Status New'] == 'Done-OLD':
        done_cases += 1
    elif row['Season'] <= 2012:
        cases_2011_12 += 1
        nationality = row['Nationality']
        this_country_code = row['Team Country Code']
        if pd.isna(this_country_code):
            country_name = 'Namibia'
        else:
            if this_country_code == 'BVI':
                country_name = 'British Virgin Islands'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value_old[leagues_value_old['Team 1 Code'] == this_country_code]['Name'].unique()
        print(i, 'WORKING', row['ORIGINAL JERSEY'], row['Nationality'], row['Match'], row['Date'])
        #print(i, threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality))

        filtered_names_match_date = filter_using_date_of_match(threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0], row['Nationality'], row['Date'], False)[1]
        # if type(filtered_names_match_date) == str:
        if filtered_names_match_date == '' or filtered_names_match_date == " ":
            if len(threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0]) == 1:
                print(i, f'noboddy found, list was{threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0]}')
                match_date_filter_said_0_guys.at[i, 'New Found Name'] = threshold_player_match(row['ORIGINAL JERSEY'], dataset_nationality)[0]
                match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done-OLD'
            else:
                print()
        else:
            print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = filtered_names_match_date
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done-OLD'
    else:
        match = row['Match']
        date = row['Date']

        ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
        if ORIGINAL_JERSEY == 'Ahmad Almahaijri':
            ORIGINAL_JERSEY = 'Ahmed Kano'
        elif ORIGINAL_JERSEY == 'Abdullah Ammar':
            ORIGINAL_JERSEY = 'Abdullah Al-Ammar'
        elif ORIGINAL_JERSEY == 'Johnson':
            if row['Nationality'] == 'USA':
                ORIGINAL_JERSEY = 'Fabian Johnson'
        elif ORIGINAL_JERSEY == 'Jones':
            ORIGINAL_JERSEY = 'Jermaine Jones'
        elif ORIGINAL_JERSEY == 'Gonzalez':
            ORIGINAL_JERSEY = 'Omar Gonzalez'
        elif ORIGINAL_JERSEY == 'inh Tin Thành':
            ORIGINAL_JERSEY = 'Thanh Tin Than'
        elif ORIGINAL_JERSEY == 'A Traoré':
            if i == 55 or i == 56:
                ORIGINAL_JERSEY = 'Adama Traoré'
        elif ORIGINAL_JERSEY == 'C Lei':
            ORIGINAL_JERSEY = 'Cheng Lam Lei'
        elif ORIGINAL_JERSEY == 'E García':
            ORIGINAL_JERSEY = 'Gabriel García'
        elif ORIGINAL_JERSEY == 'O`Shea':
            ORIGINAL_JERSEY = 'John O\'Shea'
        elif ORIGINAL_JERSEY == 'Lobjanidze':
            ORIGINAL_JERSEY = 'Ucha Lobjanidze'
        elif ORIGINAL_JERSEY == 'Alfonso González':
            ORIGINAL_JERSEY = 'Arturo González'
        elif ORIGINAL_JERSEY == 'Robinson':
            ORIGINAL_JERSEY = 'Antonee Robinson'
        elif ORIGINAL_JERSEY == 'Mc Kennie':
            ORIGINAL_JERSEY = 'Weston Mckennie'
        elif ORIGINAL_JERSEY == 'Aaronson':
            ORIGINAL_JERSEY = 'Brendan Aaronson'
        nationality = row['Nationality']
        this_country_code = row['Team Country Code']
        if pd.isna(this_country_code):
            country_name = 'Namibia'
        else:
            if this_country_code == 'BVI':
                country_name = 'British Virgin Islands'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()

        query_text = f"{ORIGINAL_JERSEY} {nationality} transfermarkt"
        names_transfermarkt_search = process_google_query(query_text)
        correct_names = []
        for name_option in names_transfermarkt_search:
            if 'Club' in name_option or 'League' in name_option:
                names_transfermarkt_search.remove(name_option)
        for name_option in names_transfermarkt_search:
            if (nationality in leagues_value[leagues_value['Name'] == name_option]['Nationality'].unique()):
                correct_names.append(name_option)
        if len(correct_names) == 1:
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
            match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = correct_names[0]
            
            print(i, 'WORKING', ORIGINAL_JERSEY, correct_names, names_transfermarkt_search)
            success_status_counter += 1
        else:
            if correct_names != []:
                #remaining_match = threshold_player_match(ORIGINAL_JERSEY, correct_names)[0]
                closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], correct_names)
            else:
                closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], names_transfermarkt_search)
            if len(closest_name) == 1:
                match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
                match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = closest_name[0]
                print(i, 'WORKING', ORIGINAL_JERSEY, closest_name, correct_names, names_transfermarkt_search)
                success_status_counter += 1
            else:
                filtered_names_match_date = filter_using_date_of_match(closest_name, row['Nationality'], date, False)[1]
                if type(filtered_names_match_date) == str:
                    if filtered_names_match_date == '':
                        
                        dataset_nationality_large = leagues_value_large[leagues_value_large['Team 1 Code'] == this_country_code]['Name'].unique()
                        list_large_set = threshold_player_match(ORIGINAL_JERSEY, dataset_nationality_large)[0]
                        if len(list_large_set) >= 1:
                            filtered_names_match_date_ii = filter_using_date_of_match(list_large_set, row['Nationality'], date, False)[1]
                            if type(filtered_names_match_date_ii) == str:
                                if filtered_names_match_date_ii == '':
                                    print(i, 'NOBODY REMAINING', nationality, match, date, ORIGINAL_JERSEY, filtered_names_match_date_ii, correct_names, names_transfermarkt_search)
                                    zero_cases += 1
                                elif filtered_names_match_date_ii.startswith('['):
                                    print(i, 'MULTIPLE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date_ii, correct_names, names_transfermarkt_search)
                                    multi_cases += 1
                                else:
                                    print(i, 'ONE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date_ii, correct_names, names_transfermarkt_search)
                                    match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
                                    match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = filtered_names_match_date_ii
                                    success_status_counter += 1
                                                            
                    elif filtered_names_match_date.startswith('['):
                        print(i, 'MULTIPLE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date, correct_names, names_transfermarkt_search)
                        multi_cases += 1
                    else:
                        print(i, 'ONE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date, correct_names, names_transfermarkt_search)
                        match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
                        match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = filtered_names_match_date
                        success_status_counter += 1

                else:
                    print(i, 'MULTIPLE REMAINING', ORIGINAL_JERSEY, filtered_names_match_date, correct_names, names_transfermarkt_search)
                    multi_cases += 1
print(done_cases, cases_2011_12, success_status_counter, multi_cases, zero_cases)


17 WORKING Saucedo Bolivia Uruguay vs Bolivia 07-Oct-11
https://www.transfermarkt.com/carlos-saucedo/profil/spieler/77940
https://www.transfermarkt.us/mauricio-saucedo/nationalmannschaft/spieler/73298/wettbewerb_id/CA11
there was no national team history cannot access local variable 'match_date_row' where it is not associated with a value
there was no national team history cannot access local variable 'match_date_row' where it is not associated with a value

23 WORKING Vargas Bolivia Venezuela vs Bolivia 15-Nov-11
https://www.transfermarkt.com/christian-vargas/nationalmannschaft/spieler/89537
https://www.transfermarkt.us/rodrigo-vargas/profil/spieler/177676
there was no national team history cannot access local variable 'match_date_row' where it is not associated with a value
reverse loop. there was no national team history time data '15/11/11' does not match format '%m/%d/%y'
reverse loop. there was no national team history time data '15/11/11' does not match format '%m/%d/%y'

24 WOR

KeyboardInterrupt: 

In [ ]:
match_date_filter_said_0_guys.at[432, 'Name(s) Found'] = 'Mamy Gervais'



match_date_filter_said_0_guys.at[402, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[402, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[406, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[406, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[411, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[411, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[412, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[412, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[413, 'Name(s) Found'] = 'Daryl Murphy'
match_date_filter_said_0_guys.at[413, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[403, 'Name(s) Found'] = 'Daryl Murphy'
match_date_filter_said_0_guys.at[403, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[392, 'Name(s) Found'] = 'Jordan Morris'
match_date_filter_said_0_guys.at[392, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[296, 'Name(s) Found'] = 'Jermaine Johnson'
match_date_filter_said_0_guys.at[296, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[299, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[299, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[225, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[225, 'Name(s) Found'] = 'Jermaine Beckford'

match_date_filter_said_0_guys.at[293, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[293, 'Name(s) Found'] = 'Jermaine Beckford'

match_date_filter_said_0_guys.at[292, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[292, 'Name(s) Found'] = 'Jermaine Beckford'


match_date_filter_said_0_guys.at[291, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[291, 'Name(s) Found'] = 'Hérculez Gómez'

match_date_filter_said_0_guys.at[295, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[295, 'Name(s) Found'] = 'Hérculez Gómez'


#293 is Jermaine Beckford, Done
#292 is Jermaine Beckford, Done
#295 is Hérculez Gómez, Done
#291 is Hérculez Gómez, Done


match_date_filter_said_0_guys.at[0, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[0, 'Name(s) Found'] = 'Arda Turan'

match_date_filter_said_0_guys.at[15, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[15, 'Name(s) Found'] = 'Abdullah Al-Ammar'

match_date_filter_said_0_guys.at[14, 'Name(s) Found'] = 'Tuan Tai Phan'

match_date_filter_said_0_guys.at[43, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[43, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[47, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[47, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[44, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[44, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[48, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[48, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[45, 'Name(s) Found'] = 'Demba Kamara'
match_date_filter_said_0_guys.at[45, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[46, 'Name(s) Found'] = 'Bubacarr Sanneh'
match_date_filter_said_0_guys.at[46, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[55, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[55, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[56, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[56, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[199, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[199, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[69, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[69, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[119, 'Name(s) Found'] = 'Mamy Gervais'
match_date_filter_said_0_guys.at[119, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[68, 'Name(s) Found'] = 'Bubacarr Sanneh'
match_date_filter_said_0_guys.at[68, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[70, 'Name(s) Found'] = 'Nito'
match_date_filter_said_0_guys.at[70, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[71, 'Name(s) Found'] = 'Raúl Fernández'
match_date_filter_said_0_guys.at[71, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[71, 'Nationality'] = 'Peru'
match_date_filter_said_0_guys.at[71, 'Team Country Code'] = 'PE'

match_date_filter_said_0_guys.at[94, 'Name(s) Found'] = 'Daryl Murphy'
match_date_filter_said_0_guys.at[94, 'Status New'] = 'Done'



match_date_filter_said_0_guys.at[117, 'Name(s) Found'] = 'Mohamed Abuaagla'
match_date_filter_said_0_guys.at[117, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[116, 'Name(s) Found'] = 'Machawe Dlamini'
match_date_filter_said_0_guys.at[116, 'Status New'] = 'Done'

#43 Ablie Jallow
#44 Musa Barrow
#45 is Demba Kamara
#46 Bubacarr Sanneh
#47 Ablie Jallow
#48 Musa Barrow

match_date_filter_said_0_guys.at[122, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[122, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[124, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[124, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[126, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[126, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[123, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[123, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[125, 'Name(s) Found'] = 'Aiden O\'Brien'
match_date_filter_said_0_guys.at[125, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[127, 'Name(s) Found'] = 'Aiden O\'Brien'
match_date_filter_said_0_guys.at[127, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[128, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[128, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[153, 'Name(s) Found'] = 'Jordan Morris'
match_date_filter_said_0_guys.at[153, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[178, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[178, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[172, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[174, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[175, 'Name(s) Found'] = 'Bùi Tấn Trường'

match_date_filter_said_0_guys.at[177, 'Name(s) Found'] = 'Khalid Ali Al-Hajjaji'

match_date_filter_said_0_guys.at[162, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[162, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[189, 'Name(s) Found'] = 'Angel Cardozo Lucena'

match_date_filter_said_0_guys.at[198, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[198, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[182, 'Name(s) Found'] = 'Momodou Ceesay'
match_date_filter_said_0_guys.at[182, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[196, 'Name(s) Found'] = 'Momodou Ceesay'
match_date_filter_said_0_guys.at[196, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[181, 'Name(s) Found'] = 'Sanna Nyassi'
match_date_filter_said_0_guys.at[181, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[180, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[180, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[183, 'Name(s) Found'] = 'José María Giménez'
match_date_filter_said_0_guys.at[183, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[183, 'Nationality'] = 'Uruguay'
match_date_filter_said_0_guys.at[183, 'Team Country Code'] = 'UY'

match_date_filter_said_0_guys.at[195, 'Name(s) Found'] = 'Bubacarr Sanneh'
match_date_filter_said_0_guys.at[195, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[202, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[206, 'Name(s) Found'] = 'Bùi Tấn Trường'
match_date_filter_said_0_guys.at[206, 'Name(s) Found'] = 'Mohamed Fasal'
match_date_filter_said_0_guys.at[200, 'Name(s) Found'] = 'Abdoulaye Diallo'

match_date_filter_said_0_guys.at[223, 'Name(s) Found'] = 'Jermaine Johnson'
match_date_filter_said_0_guys.at[224, 'Name(s) Found'] = 'Ryan Johnson'
match_date_filter_said_0_guys.at[227, 'Name(s) Found'] = 'Eddie Johnson'
match_date_filter_said_0_guys.at[230, 'Name(s) Found'] = 'Eddie Johnson'
match_date_filter_said_0_guys.at[236, 'Name(s) Found'] = 'Eddie Johnson'

match_date_filter_said_0_guys.at[241, 'Name(s) Found'] = 'Jermaine Johnson'

match_date_filter_said_0_guys.at[302, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[302, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[305, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[305, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[309, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[309, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[304, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[304, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[306, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[306, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[317, 'Name(s) Found'] = 'Adama Traoré'
match_date_filter_said_0_guys.at[317, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[300, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[300, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[303, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[303, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[307, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[307, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[308, 'Name(s) Found'] = 'Modou Barrow'
match_date_filter_said_0_guys.at[308, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[311, 'Name(s) Found'] = 'Musa Barrow'
match_date_filter_said_0_guys.at[311, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[310, 'Name(s) Found'] = 'Modou Barrow'
match_date_filter_said_0_guys.at[310, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[319, 'Name(s) Found'] = 'Mamy Gervais'
match_date_filter_said_0_guys.at[319, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[312, 'Name(s) Found'] = 'Ablie Jallow'
match_date_filter_said_0_guys.at[312, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[313, 'Name(s) Found'] = 'Modou Barrow'
match_date_filter_said_0_guys.at[313, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[315, 'Name(s) Found'] = 'Pa Modou'
match_date_filter_said_0_guys.at[315, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[318, 'Name(s) Found'] = 'Mohamed Abuaagla'
match_date_filter_said_0_guys.at[318, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[316, 'Name(s) Found'] = 'Momodou Ceesay'
match_date_filter_said_0_guys.at[316, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[324, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[324, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[330, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[330, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[332, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[332, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[335, 'Name(s) Found'] = 'Arda Turan'
match_date_filter_said_0_guys.at[335, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[325, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[325, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[326, 'Name(s) Found'] = 'Shane Long'
match_date_filter_said_0_guys.at[326, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[327, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[327, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[328, 'Name(s) Found'] = 'Kevin Long'
match_date_filter_said_0_guys.at[328, 'Status New'] = 'Done'

match_date_filter_said_0_guys.at[336, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[336, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[337, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[337, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[338, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[338, 'Status New'] = 'Done'
match_date_filter_said_0_guys.at[339, 'Name(s) Found'] = 'Alexander González'
match_date_filter_said_0_guys.at[339, 'Status New'] = 'Done'


match_date_filter_said_0_guys.at[340, 'Name(s) Found'] = 'James Rodriguez'
match_date_filter_said_0_guys.at[340, 'Nationality'] = 'Colombia'
match_date_filter_said_0_guys.at[340, 'Team Country Code'] = 'CO'

match_date_filter_said_0_guys.at[350, 'Name(s) Found'] = 'Mohamed Fasal'
match_date_filter_said_0_guys.at[354, 'Name(s) Found'] = 'Khalid Ali Al Hajjaji'



for i in range(0, 438):
    if match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Luis Puma Rodríguez':
        match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        match_date_filter_said_0_guys.at[i, 'Name(s) Found'] = 'Puma'

In [ ]:
#leagues_value_large[leagues_value_large['Name'] == 'Mamy Gervais']

In [7]:
#247 Wilson Palacios Honduras HN
match_date_filter_said_0_guys.at[247, 'New Found Name'] = 'Wilson Palacios'
match_date_filter_said_0_guys.at[247, 'Nationality'] = 'Honduras'
match_date_filter_said_0_guys.at[247, 'Team Country Code'] = 'HN'
#248 Isidro Gutierrez
match_date_filter_said_0_guys.at[248, 'New Found Name'] = 'Isidro Gutierrez'
match_date_filter_said_0_guys.at[248, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[248, 'Team Country Code'] = 'SV'
#250, Mexico, MX, Francisco Rodriguez
match_date_filter_said_0_guys.at[250, 'New Found Name'] = 'Francisco Rodríguez'
match_date_filter_said_0_guys.at[250, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[250, 'Team Country Code'] = 'SV'
#251 Mexico MX Jorge Torres Nilo
match_date_filter_said_0_guys.at[251, 'New Found Name'] = 'Jorge Torres Nilo'
match_date_filter_said_0_guys.at[251, 'Nationality'] = 'Mexico'
match_date_filter_said_0_guys.at[251, 'Team Country Code'] = 'MX'
#252 El Salvador SV Ramón Sánchez
match_date_filter_said_0_guys.at[252, 'New Found Name'] = 'Ramón Sánchez'
match_date_filter_said_0_guys.at[252, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[252, 'Team Country Code'] = 'SV'
#258 Gregory Richardson Guyana GY
match_date_filter_said_0_guys.at[250, 'New Found Name'] = 'Gregory Richardson'
match_date_filter_said_0_guys.at[250, 'Nationality'] = 'Guyana'
match_date_filter_said_0_guys.at[250, 'Team Country Code'] = 'GY'
#280, Mexico, MX, Francisco Rodriguez
match_date_filter_said_0_guys.at[280, 'New Found Name'] = 'Francisco Rodríguez'
match_date_filter_said_0_guys.at[280, 'Nationality'] = 'El Salvador'
match_date_filter_said_0_guys.at[280, 'Team Country Code'] = 'SV'


In [9]:
for i in range(0, len(match_date_filter_said_0_guys)):
    if match_date_filter_said_0_guys.at[i, 'Status New'] != 'Done':
        if match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Vargas':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Christian Vargas'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        elif match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Cummings':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Shaun Cummings'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        elif match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Saucedo':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Mauricio Saucedo'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        elif match_date_filter_said_0_guys.at[i, 'ORIGINAL JERSEY'] == 'Johnson' and match_date_filter_said_0_guys.at[i, 'Nationality'] == 'USA':
            match_date_filter_said_0_guys.at[i, 'New Found Name'] = 'Fabian Johnson'
            match_date_filter_said_0_guys.at[i, 'Status New'] = 'Done'
        

In [12]:
match_date_filter_said_0_guys[match_date_filter_said_0_guys['Status New'] != 'Done']#.tail(7)

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,...,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Status New,New Found Name
242,242,242,242,58369,69,3897,"World Cup 2014, Qualifiers, CONCACAF",Cuba vs Canada,08-Jun-12,Cuba,...,"['Sánder Fernández', 'Onel Hernández']",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Sánder Fernández', 'Onel Hernández']",Fernandez,multiple,NaN,NaN
243,243,243,243,58390,90,3898,"World Cup 2014, Qualifiers, CONCACAF",Mexico vs Guyana,08-Jun-12,Mexico,...,"['Érick Torres', 'Jairo Torres']",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Érick Torres', 'Jairo Torres']",Torres,multiple,NaN,NaN
249,249,249,249,58477,177,3902,"World Cup 2014, Qualifiers, CONCACAF",Jamaica vs Guatemala,08-Jun-12,Jamaica,...,"['Kevin Stewart', 'Trivante Stewart']",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Kevin Stewart', 'Trivante Stewart']",Stewart,multiple,NaN,NaN
253,253,253,253,58568,268,3906,"World Cup 2014, Qualifiers, CONCACAF",Guatemala vs USA,12-Jun-12,Guatemala,...,"['Rubén Morales', 'Rafael Morales', 'José Mora...",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Rubén Morales', 'Rafael Morales', 'José Mora...",Morales,multiple,NaN,NaN
257,257,257,257,58600,300,3907,"World Cup 2014, Qualifiers, CONCACAF",Guyana vs Costa Rica,12-Jun-12,Costa Rica,...,"['Ariel Rodriguez', 'Osvaldo Rodriguez']",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Ariel Rodriguez', 'Osvaldo Rodriguez']",Rodrigues,multiple,NaN,NaN
259,259,259,259,58608,308,3908,"World Cup 2014, Qualifiers, CONCACAF",Panama vs Cuba,12-Jun-12,Panama,...,"['Álex Rodríguez', 'Carlos Rodríguez', 'Jhamal...",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Álex Rodríguez', 'Carlos Rodríguez', 'Jhamal...",Rodríguez,multiple,NaN,NaN
260,260,260,260,58616,316,3908,"World Cup 2014, Qualifiers, CONCACAF",Panama vs Cuba,12-Jun-12,Panama,...,"['Gabriel Gómez', 'Martín Gómez']",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Gabriel Gómez', 'Martín Gómez']",Gómez,multiple,NaN,NaN
261,261,261,261,58661,361,3910,"World Cup 2014, Qualifiers, CONCACAF",Jamaica vs USA,07-Sep-12,Jamaica,...,"['Jason Johnson', 'Jermaine Johnson', 'Daniel ...",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Jason Johnson', 'Jermaine Johnson', 'Daniel ...",Johnson,multiple,NaN,NaN
265,265,265,265,58708,408,3912,"World Cup 2014, Qualifiers, CONCACAF",Canada vs Panama,07-Sep-12,Panama,...,"['Edgar Góndola', 'Edgardo Fariña']",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Edgar Góndola', 'Edgardo Fariña']",Edgar,multiple,NaN,NaN
266,266,266,266,58745,445,3914,"World Cup 2014, Qualifiers, CONCACAF",Costa Rica vs Mexico,07-Sep-12,Costa Rica,...,"['Ariel Rodríguez', 'Osvaldo Rodríguez']",0.0,True,FILTERING USING MATCH DATE LEFT 0 RESULTS. BEF...,False,"['Ariel Rodríguez', 'Osvaldo Rodríguez']",Rodriguez,multiple,NaN,NaN


In [11]:
match_date_filter_said_0_guys.to_csv('updated_match_date_filter_said_0_guys_iiI.csv')

In [ ]:
for name_option in names_transfermarkt_search:
    if name_option in dataset_nationality:
        print(name_option)

In [ ]:
#v1 of logic
for i in range(0, 438):
    
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = match_date_filter_said_0_guys.loc[index]
    # Extract relevant columns

    if row['Status New'] == 'Done':
        0==0
    else:

        match = row['Match']
        date = row['Date']
        ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
        nationality = row['Nationality']
        this_country_code = row['Team Country Code']
        if pd.isna(this_country_code):
            country_name = 'Namibia'
        else:
            if this_country_code == 'BVI':
                country_name = 'British Virgin Islands'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()

        NAMES_FOUND = row['Name(s) Found']
        FOUND_NAME = row['Name']
        
        if type(FOUND_NAME) == list:
            if NAMES_FOUND.startswith('['):
                for name in eval(NAMES_FOUND):
                    FOUND_NAME.append(name)
            else:
                FOUND_NAME.append(NAMES_FOUND)
            found_names = FOUND_NAME
        else:
            found_names = [FOUND_NAME, NAMES_FOUND]
        closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)

        threshold_last_name = threshold_player_match(ORIGINAL_JERSEY.split(' ')[-1], dataset_nationality)[0]
        if closest_name != []:
            intersection = [name for name in closest_name if name in found_names]
        else:
            intersection = [name for name in found_names if name in threshold_last_name]

        if row['Name(s) Found'].startswith('['):
            candidate_set = eval(row['Name(s) Found'])
        else:
            candidate_set = row['Name(s) Found']
        print(f"**{i}**")
        if type(intersection) == list:
            print(i, ORIGINAL_JERSEY, row['Nationality'], (intersection), closest_name, found_names, threshold_last_name)
        #filtered_names_match_date = filter_using_date_of_match(candidate_set, row['Nationality'], date, False)[1]
        #if type(filtered_names_match_date) == str:
            #print(i, ORIGINAL_JERSEY, closest_name, found_names)
            #print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
            #wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
            #wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
            #success_status_counter += 1

## New TM CSV

In [ ]:
new_tm_csv = load_csv_dataset('MarketValues.csv')

new_tm_csv['Season'].value_counts()